In [1]:
import numpy as np
import pandas as pd
import os

from common import OUTPUTPATH
from models import rule_fit_pcc
import data

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)


<ipython-input-1-ea58b87889e3>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


This notebook refit the models in the all dataset

Fit all data takes ~10 mim under 2.6 GHz 6-Core Intel Core i7

In [2]:
STATE = np.random.RandomState(seed=1000)
rule_fit_pcc.fit(data.x1, data.y)

In [11]:
fitted_model = rule_fit_pcc.fitted_
mophology = ['Sphere', 'Worm', 'Vesicle', 'Other']
for i in range(len(fitted_model)):
    print(mophology[i], 'Cs:', fitted_model[i].model.Cs)

Sphere Cs: [1]
Worm Cs: [1]
Vesicle Cs: [1]
Other Cs: [32]


Get the individual morphology rules

In [12]:
all_columns = data.predictors + data.targets
rule_sphere = rule_fit_pcc.fitted_[0].get_rules(all_columns)[['rule','type','coef','importance']]
rule_sphere.to_csv(os.path.join(OUTPUTPATH, 'Sphere_rules.csv'), index=False)
rule_sphere

,rule,type,coef,importance
1169,core_mw_total> 16317.3247 & conc> 16.1 & mon_core_psa> 40.85 & corona_mw_total<= 12760.5298 & core_mw_total<= 62496.0391,rule,-1.802035,0.824182
800,dp_core> 102.5 & corona_mv_total<= 84.5716 & mon_core_apol<= 22.8716 & conc<= 16.5 & charged<= 0.5 & conc> 6.25,rule,-2.065141,0.763292
446,conc> 16.1 & corona_mv_total> 42.9745 & dp_core> 209.5 & mon_corona_psa> 65.0,rule,-2.025070,0.748481
620,corona_mv_total> 42.9745 & clogp_core> 0.665 & ratio_vol<= 0.601,rule,1.391977,0.605184
846,dp_corona> 62.5 & dp_core<= 199.0,rule,2.292133,0.582459
1287,clogp_core<= 1.37 & dp_corona<= 64.5 & mon_corona_apol<= 26.9812 & ratio_mass<= 0.2165,rule,-1.557882,0.524969
1325,core_mw_total<= 59921.1914 & dp_core> 102.5 & ratio_mass> 0.2846 & core_mw_total> 17289.5996 & conc> 16.5,rule,2.339009,0.462807
860,conc<= 16.0 & dp_corona> 54.5 & mon_core_mw> 143.9765,rule,1.056787,0.456978
133,corona_mw_total<= 6338.1602 & ratio_mass<= 0.4342 & conc> 6.25,rule,-1.090694,0.452116
1123,ratio_mass<= 0.325 & mon_core_mw<= 145.3096 & dp_corona> 27.5 & corona_mw_total<= 15743.5601,rule,-0.873231,0.434337


In [13]:
rule_worm = rule_fit_pcc.fitted_[1].get_rules(all_columns)[['rule','type','coef','importance']]
rule_worm.to_csv(os.path.join(OUTPUTPATH, 'Worm_rules.csv'), index=False)
rule_worm

,rule,type,coef,importance
809,mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_corona<= 0.262,rule,-1.742955,0.836202
848,ratio_vol> 0.1316 & ratio_vol> 0.1677 & dp_corona> 28.5 & dp_core<= 251.5 & dp_corona<= 41.5,rule,1.904156,0.773076
1116,dp_core> 139.5 & clogp_core> 0.895 & ratio_vol<= 0.3447 & clogp_core<= 1.1225 & core_mv_total<= 354.9075,rule,1.337368,0.623003
624,corona_mw_total> 8883.8848 & ratio_mass<= 0.4161 & core_mv_total<= 327.7388 & conc> 12.75,rule,1.706850,0.617883
1445,core_mv_total<= 414.75 & ratio_mass> 0.1285 & dp_corona> 77.5 & corona_mv_total<= 109.4456 & core_mv_total> 192.5625,rule,1.838302,0.556458
1470,ratio_mass> 0.3093 & ratio_vol> 0.2538 & sphere> 0.5,rule,-1.443319,0.505057
1367,sphere> 0.5 & core_mv_total<= 298.0275 & clogp_core<= 1.14 & clogp_core<= 1.11 & dp_core<= 203.0 & core_mv_total> 89.3223 & mon_core_mw> 136.06 & ratio_vol<= 0.4203,rule,-1.300289,0.485379
327,conc> 11.25 & dp_corona<= 77.5 & clogp_corona> -5.23 & dp_core> 99.5,rule,-1.000511,0.480006
847,mon_core_psa<= 46.7805 & conc<= 24.0 & corona_mw_total> 5985.855,rule,-0.929479,0.442428
1182,mon_core_psa<= 46.7805 & core_mw_total> 21315.8496 & core_mv_total> 131.1768,rule,-0.897559,0.413668


In [14]:
rule_vesicle = rule_fit_pcc.fitted_[2].get_rules(all_columns)[['rule','type','coef','importance']]
rule_vesicle.to_csv(os.path.join(OUTPUTPATH, 'Vesicle_rules.csv'), index=False)
rule_vesicle

,rule,type,coef,importance
886,corona_mw_total<= 5806.375 & corona_mw_total> 3982.275 & mon_core_psa<= 46.7729,rule,1.925786,0.739207
820,mon_core_psa<= 46.5679 & temp> 53.0 & ratio_mass<= 0.251 & sphere<= 0.5 & ratio_vol> 0.0522,rule,1.620464,0.688824
501,temp> 65.0 & clogp_corona> -5.23 & mon_core_mw> 136.8175 & ratio_mass<= 0.2777 & sphere<= 0.5 & ratio_vol> 0.0522,rule,1.533366,0.673611
696,mon_corona_apol<= 23.6109 & clogp_core<= 1.38 & ratio_vol<= 0.2191 & charged<= 0.5,rule,1.173012,0.495625
802,temp> 65.0 & ratio_vol<= 0.2574 & ratio_vol<= 0.2599 & corona_mw_total> 8503.8901 & ratio_mass<= 0.2654 & core_mw_total<= 58128.9121,rule,1.258304,0.474233
1276,ratio_vol<= 0.3723 & clogp_corona> -5.23 & core_mw_total<= 33527.2109 & dp_corona<= 34.5 & ph<= 3.85,rule,2.139021,0.380161
571,ratio_vol<= 0.1768 & core_mw_total<= 179112.7969,rule,0.731168,0.351648
1147,conc<= 12.5 & corona_mw_total<= 14063.4102 & worm<= 0.5 & clogp_core<= 1.38 & clogp_core> 0.665 & ratio_mass> 0.0551 & corona_mv_total> 25.3733 & sphere<= 0.5 & mon_core_apol> 21.994,rule,1.338902,0.349327
473,salt<= 0.15 & mon_core_apol<= 22.7739 & ph> 4.25 & worm<= 0.5 & sphere<= 0.5 & core_mw_total<= 152724.7969 & mon_core_apol> 22.0607 & ratio_mass<= 0.2669,rule,0.980580,0.343131
656,temp> 65.0 & clogp_corona> -5.23 & clogp_core<= 1.38 & conc> 11.5 & ratio_mass<= 0.2467 & mon_core_mw> 136.06,rule,0.860882,0.333349


In [15]:
rule_other = rule_fit_pcc.fitted_[3].get_rules(all_columns)[['rule','type','coef','importance']]
rule_other.to_csv(os.path.join(OUTPUTPATH, 'Other_rules.csv'), index=False)
rule_other

,rule,type,coef,importance
761,sphere> 0.5,rule,-2.900708,1.431861
23,sphere,linear,-2.604619,1.298708
1033,salt<= 0.15 & temp> 47.0 & clogp_core<= 1.4 & core_mw_total> 9221.1196 & sphere<= 0.5 & corona_mv_total<= 107.5755,rule,-2.600697,1.271863
771,mon_corona_psa<= 47.97 & mon_corona_apol> 23.9055 & mon_core_psa<= 46.5755 & sphere<= 0.5,rule,4.210728,0.533617
215,clogp_corona<= 0.605 & mon_corona_mv> 0.7838 & sphere<= 0.5 & corona_mv_total> 107.5755 & mon_corona_apol<= 33.9505,rule,5.415209,0.487267
245,conc> 18.75 & mon_corona_mv<= 0.7838 & sphere<= 0.5 & core_mw_total<= 33858.7998,rule,3.279666,0.415625
932,clogp_corona<= 0.64 & salt<= 0.15 & corona_mw_total<= 14063.4102 & clogp_core<= 1.4 & sphere<= 0.5,rule,-0.748811,0.366841
630,conc> 18.75 & core_mw_total<= 32418.0 & clogp_corona> 0.125 & conc> 6.25,rule,2.593399,0.366686
309,mon_corona_mv<= 0.7838 & ratio_mass> 0.1285 & conc> 17.5 & sphere<= 0.5,rule,2.311414,0.292921
46,clogp_corona> -1.0511,rule,-0.562887,0.266411


Get the Top5 morphology rules

In [16]:
# get top 5:
sphere_top5_rule = rule_sphere.iloc[:5, ]
worm_top5_rule = rule_worm.iloc[:5, ]
vesicle_top5_rule = rule_vesicle.iloc[:5, ]

sphere_top5_rule.reset_index(drop=True, inplace=True)
worm_top5_rule.reset_index(drop=True, inplace=True)
vesicle_top5_rule.reset_index(drop=True, inplace=True)

top5_rule = pd.concat([sphere_top5_rule, worm_top5_rule, vesicle_top5_rule], axis=1)
upper_columns = ['Sphere', 'Worm', 'Vesicle']
lower_columns = ['rule', 'type', 'coef', 'importance']

top5_rule.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
top5_rule.to_csv(os.path.join(OUTPUTPATH, 'Combine_Top5_Rules.csv'), index=False)
top5_rule


Phase                                                                                                                       Sphere  \
Property                                                                                                                      rule   
0         core_mw_total> 16317.3247 & conc> 16.1 & mon_core_psa> 40.85 & corona_mw_total<= 12760.5298 & core_mw_total<= 62496.0391   
1         dp_core> 102.5 & corona_mv_total<= 84.5716 & mon_core_apol<= 22.8716 & conc<= 16.5 & charged<= 0.5 & conc> 6.25            
2         conc> 16.1 & corona_mv_total> 42.9745 & dp_core> 209.5 & mon_corona_psa> 65.0                                              
3         corona_mv_total> 42.9745 & clogp_core> 0.665 & ratio_vol<= 0.601                                                           
4         dp_corona> 62.5 & dp_core<= 199.0                                                                                          

Phase                                \
Property  type      coef importance   
0         rule -1.802035  0.824182    
1         rule -2.065141  0.763292    
2         rule -2.025070  0.748481    
3         rule  1.391977  0.605184    
4         rule  2.292133  0.582459    

Phase                                                                                                                     Worm  \
Property                                                                                                                  rule   
0         mon_core_psa<= 46.7805 & conc<= 11.25 & clogp_corona<= 0.262                                                           
1         ratio_vol> 0.1316 & ratio_vol> 0.1677 & dp_corona> 28.5 & dp_core<= 251.5 & dp_corona<= 41.5                           
2         dp_core> 139.5 & clogp_core> 0.895 & ratio_vol<= 0.3447 & clogp_core<= 1.1225 & core_mv_total<= 354.9075               
3         corona_mw_total> 8883.8848 & ratio_mass<= 0.4161 & core_mv_total<= 327.7388 & conc> 12.75                              
4         core_mv_total<= 414.75 & ratio_mass> 0.1285 & dp_corona> 77.5 & corona_mv_total<= 109.4456 & core_mv_total> 192.5625   

Phase                                \
Property  type      coef importance   
0         rule -1.742955  0.836202    
1         rule  1.904156  0.773076    
2         rule  1.337368  0.623003    
3         rule  1.706850  0.617883    
4         rule  1.838302  0.556458    

Phase                                                                                                                                  Vesicle  \
Property                                                                                                                                  rule   
0         corona_mw_total<= 5806.375 & corona_mw_total> 3982.275 & mon_core_psa<= 46.7729                                                        
1         mon_core_psa<= 46.5679 & temp> 53.0 & ratio_mass<= 0.251 & sphere<= 0.5 & ratio_vol> 0.0522                                            
2         temp> 65.0 & clogp_corona> -5.23 & mon_core_mw> 136.8175 & ratio_mass<= 0.2777 & sphere<= 0.5 & ratio_vol> 0.0522                      
3         mon_corona_apol<= 23.6109 & clogp_core<= 1.38 & ratio_vol<= 0.2191 & charged<= 0.5                                                     
4         temp> 65.0 & ratio_vol<= 0.2574 & ratio_vol<= 0.2599 & corona_mw_total> 8503.8901 & ratio_mass<= 0.2654 & core_mw_total<= 58128.9121   

Phase                                
Property  type      coef importance  
0         rule  1.925786  0.739207   
1         rule  1.620464  0.688824   
2         rule  1.533366  0.673611   
3         rule  1.173012  0.495625   
4         rule  1.258304  0.474233